In [1]:
import pandas as pd
import datetime
import time
import requests
import numpy as np

In [2]:
def get_sudden_rising_df(idx):
    list_columns = [
        "종목명", "등락률", "현재가", "전일비", "시가", "고가", "저가", "거래량",
    ]
    list_int = [
        "현재가", "전일비", "시가", "고가", "저가", "거래량"
    ]
    
    url = f"https://finance.naver.com/sise/sise_low_up.naver?sosok={idx}"
    list_df = pd.read_html(url, encoding="euc-kr")

    df_sudden_rising = list_df[1]
    df_sudden_rising = df_sudden_rising[~df_sudden_rising["N"].isna()]
    
    for col in list_int:
        df_sudden_rising[col] = df_sudden_rising[col].astype(int)
    
    return df_sudden_rising[list_columns]

In [4]:
df_sudden_rising = pd.concat([get_sudden_rising_df(0), get_sudden_rising_df(1)])

df_sudden_rising.to_csv("sudden_rising.csv", encoding="utf-8-sig", index=False)

/var/folders/l3/676z_n851v1c6zknb1mnrgtr0000gn/T/ipykernel_1034/611320729.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sudden_rising[col] = df_sudden_rising[col].astype(int)


In [13]:
# Load previous data
df_sudden_rising = pd.read_csv("sudden_rising.csv", encoding="utf-8-sig")
df_sudden_rising

,종목명,등락률,현재가,전일비,시가,고가,저가,거래량
0,이엔플러스,30.28%,5980,1375,4800,5980,4590,32436573
1,미래아이앤지,25.11%,563,113,456,585,450,62404742
2,KH 필룩스,22.12%,3395,700,2890,3475,2780,48476237
3,한신기계,17.54%,10050,1580,8550,10350,8550,56338393
4,동양철관,16.10%,1370,165,1210,1470,1180,49745231
...,...,...,...,...,...,...,...,...
195,이엠넷,5.73%,6640,290,6280,6650,6280,89262
196,바이넥스,5.70%,16700,800,15900,17000,15800,522568
197,한국정보공학,5.68%,5210,225,4985,5210,4930,210629
198,제일전기공업,5.67%,15850,500,15350,16300,15000,104030


In [29]:
# 최근 행방
df_day_summary = pd.read_csv("../bs_analysis/day_summary.csv", encoding="utf-8-sig")
df_day_summary["DT"].unique()

array(['2022-04-04', '2022-04-01', '2022-03-31', '2022-03-30',
       '2022-03-29', '2022-03-28', '2022-03-25'], dtype=object)

In [5]:
list_ordered_key = ["CD","DT"]
df_day_summary.sort_values(by=list_ordered_key)[["NM", "DT", "FRGN_RT"]]

,NM,DT,FRGN_RT
2488,메리츠화재,2022-03-25,11.62
2144,메리츠화재,2022-03-28,11.61
1780,메리츠화재,2022-03-29,11.63
1395,메리츠화재,2022-03-30,11.62
982,메리츠화재,2022-03-31,11.61
...,...,...,...
735,잉글우드랩,2022-04-01,10.92
298,잉글우드랩,2022-04-04,11.11
993,미투젠,2022-03-31,7.99
561,미투젠,2022-04-01,8.13


In [14]:
df_sudden_rising = df_sudden_rising.rename(columns={"종목명": "NM", "등락률": "UPDN_DESC"})
df_sudden_rising.columns
last_dt = df_day_summary["DT"].max()
lasta_dt = "2022-04-01"
df_day_summary = df_day_summary[df_day_summary["DT"] == last_dt]

df_base = pd.merge(
    df_sudden_rising, df_day_summary,
    how="inner",
    on=["NM"]
)

df_base["REMAIN_VOL"] = df_base["REMAIN_BUY"] - df_base["REMAIN_SELL"]
df_base["UPDN_RT"] = df_base["UPDN_DESC"].apply(lambda X: float(X.replace("%","")))
df_base[["NM", "UPDN_RT", "END_PRC", "PRC_MEDIAN", "VOL", "VOL_MEDIAN", "REMAIN_SELL", "REMAIN_BUY", "REMAIN_VOL", "REMAIN_BUY_AMOUNT"]]

,NM,UPDN_RT,END_PRC,PRC_MEDIAN,VOL,VOL_MEDIAN,REMAIN_SELL,REMAIN_BUY,REMAIN_VOL,REMAIN_BUY_AMOUNT
0,남선알미우,13.64,75000,30575,123549,66204,3922,97,-3825,-204.0
1,이구산업,8.42,5150,4715,9478957,2954219,341667,95139,-246528,-299.0
2,크래프톤,8.35,298500,278250,503090,160292,43963,19139,-24824,-6882.0
3,남선알미늄,7.69,4060,3522,9496797,12663735,126982,493116,366134,846.0
4,한솔로지스틱스,7.09,4380,3637,4487123,205495,105788,91792,-13996,-88.0
5,한국전자홀딩스,6.61,1935,1845,5998850,347222,128974,143150,14176,-91.0
6,팜젠사이언스,6.44,9420,9420,153926,84601,21495,5101,-16394,-62.0
7,한일철강,5.81,4095,4020,176405,208845,13996,3545,-10451,-31.0
8,모나미,5.08,5380,4540,14443423,500930,76678,46267,-30411,-202.0
9,제일기획,4.61,24950,22975,723759,193895,114400,11746,-102654,-2284.0


In [25]:
# 지난 거래일에 어떤 수치였을때 오늘 올랐을까?
# 최근 10일 거래량 중간값 비교
df_vol = df_base[["NM", "UPDN_RT", "VOL", "VOL_MEDIAN"]]
df_vol["VOL_RT"] = round(df_base["VOL"] / df_base["VOL_MEDIAN"], 2)
df_vol[(df_vol["VOL_RT"] > 10.0) & (df_vol["VOL"] > 2500000)].sort_values(by=["VOL_RT"], ascending=False)
# 최근 10일 종가 중간값 비교
# 매수대기
# 외국인 비율
# df_day_summary[df_day_summary["CD"] == "A078600"]

/var/folders/l3/676z_n851v1c6zknb1mnrgtr0000gn/T/ipykernel_722/1370541341.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vol["VOL_RT"] = round(df_base["VOL"] / df_base["VOL_MEDIAN"], 2)


,NM,UPDN_RT,VOL,VOL_MEDIAN,VOL_RT
13,비올,10.37,14092386,288075,48.92
8,모나미,5.08,14443423,500930,28.83
26,오하임아이엔티,6.07,13273265,552933,24.01
4,한솔로지스틱스,7.09,4487123,205495,21.84
5,한국전자홀딩스,6.61,5998850,347222,17.28
10,스튜디오산타클로스,25.57,3101525,197411,15.71


In [26]:
df_day_summary

,DT,CD,NM,FRGN_RT,END_PRC,VOL,PRC_AVG_5,VOL_AVG_5,PRC_AVG_10,VOL_AVG_10,PRC_MEDIAN,VOL_MEDIAN,REMAIN_SELL,REMAIN_BUY,REMAIN_BUY_AMOUNT,DAY_SUMMARY
0,2022-04-04,A098120,마이크로컨텍솔,14.75,10700,112227,11020,128651,10995,93235,11025,77339,15408,23142,-56.0,"{'시가': '10550', '고가': '10700', '저가': '10300', ..."
1,2022-04-04,A095570,AJ네트웍스,4.94,6630,290863,6372,565588,6174,354788,6135,102746,4022,25112,75.0,"{'시가': '6740', '고가': '6880', '저가': '6620', '52..."
2,2022-04-04,A282330,BGF리테일,30.89,179000,30057,175900,34377,177550,33972,179000,33269,6616,3189,136.0,"{'시가': '177500', '고가': '182000', '저가': '177000..."
3,2022-04-04,A000995,DB하이텍1우,1.07,99700,420,99160,687,96940,429,95900,295,219,77,-17.0,"{'시가': '98600', '고가': '100000', '저가': '98500',..."
4,2022-04-04,A000990,DB하이텍,24.73,72900,436675,74920,665789,75130,769711,75450,730432,25367,82084,815.0,"{'시가': '73400', '고가': '73400', '저가': '72200', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,2022-04-04,A298000,효성화학,7.39,263500,16079,264000,20640,259450,16375,257750,15264,1083,3367,-2.0,"{'시가': '267000', '고가': '267500', '저가': '260000..."
436,2022-04-04,A081660,휠라홀딩스,40.17,32200,404608,31620,289344,31545,442219,31500,383451,39653,36035,63.0,"{'시가': '31500', '고가': '32300', '저가': '31250', ..."
437,2022-04-04,A290270,휴네시온,2.09,5980,144672,5844,63934,5769,59902,5790,49976,8717,8712,11.0,"{'시가': '5970', '고가': '6050', '저가': '5810', '52..."
438,2022-04-04,A205470,휴마시스,4.77,17600,696547,18030,1033477,18490,1547708,18650,1399404,71314,126654,773.0,"{'시가': '17400', '고가': '17800', '저가': '17250', ..."


In [51]:
# 처음 마지막 값
def make_first_median_value(df_base, col):
    df_first_last = df_base.groupby(["NM"]).agg(
        VOL=(col, "first"), 
        MEDIAN_VAL=("VOL_MEDIAN", "first")).reset_index()
    
    df_first_last["UPDN_RT"] = round((df_first_last["VOL"] / df_first_last["MEDIAN_VAL"]), 2)
    df_first_last = df_first_last[(df_first_last["UPDN_RT"] > 15.0) & (df_first_last["VOL"] > 2000000)]

    return df_first_last.sort_values(by=["UPDN_RT"], ascending=False).fillna(0.0)

df_vol = make_first_median_value(df_day_summary, "VOL")

In [48]:
df_day_summary[df_day_summary["NM"] == "한국전자홀딩스"]

,DT,CD,NM,FRGN_RT,END_PRC,VOL,PRC_AVG_5,VOL_AVG_5,PRC_AVG_10,VOL_AVG_10,PRC_MEDIAN,VOL_MEDIAN,REMAIN_SELL,REMAIN_BUY,REMAIN_BUY_AMOUNT,DAY_SUMMARY
408,2022-04-04,A006200,한국전자홀딩스,7.84,1935,5998850,1873,1598212,1854,935114,1845,347222,128974,143150,-91.0,"{'시가': '1820', '고가': '2035', '저가': '1815', '52..."


In [54]:
df_frgn = df_day_summary[["DT","CD","NM","FRGN_RT"]]
df_result = pd.merge(
    df_vol, df_frgn,
    how="inner",
    on=["NM"]
)

df_result

,NM,VOL,MEDIAN_VAL,UPDN_RT,DT,CD,FRGN_RT
0,비올,14092386,288075,48.92,2022-04-04,A335890,2.36
1,비올,14092386,288075,48.92,2022-04-01,A335890,1.86
2,비올,14092386,288075,48.92,2022-03-31,A335890,1.82
3,비올,14092386,288075,48.92,2022-03-30,A335890,1.81
4,비올,14092386,288075,48.92,2022-03-29,A335890,1.81
5,비올,14092386,288075,48.92,2022-03-28,A335890,1.85
6,비올,14092386,288075,48.92,2022-03-25,A335890,1.83
7,모나미,14443423,500930,28.83,2022-04-04,A005360,3.10
8,모나미,14443423,500930,28.83,2022-04-01,A005360,3.06
9,모나미,14443423,500930,28.83,2022-03-31,A005360,3.12


In [66]:
def check_frgn_rising(df_base):
    df_base = df_base.sort_values(by=["DT"])
    df_base["DIFF"] = df_base.FRGN_RT.diff()
    return df_base.tail(1)
    
list_nm = df_result["NM"].unique()
list_df = []
for nm in list_nm:
    list_df.append(check_frgn_rising(df_result[df_result["NM"] == nm].head(2)))
    
df_target = pd.concat(list_df)
df_target[df_target["DIFF"] > 0.0]

,NM,VOL,MEDIAN_VAL,UPDN_RT,DT,CD,FRGN_RT,DIFF
0,비올,14092386,288075,48.92,2022-04-04,A335890,2.36,0.50
7,모나미,14443423,500930,28.83,2022-04-04,A005360,3.10,0.04
